# Imports

In [2]:
import math

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pio.renderers.default = 'vscode'
pio.templates.default = 'plotly'

import plotly.figure_factory as ff
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, auc, classification_report,
                             confusion_matrix, roc_auc_score, roc_curve)
from sklearn.preprocessing import label_binarize

## load data

In [4]:
import pandas as pd

calendar_total_data = pd.read_parquet("../Data/total data/raw data parquet/calendar.parquet")

reviews_total_data  = pd.concat([pd.read_parquet("../Data/total data/raw data parquet/reviews_part_01.parquet"),pd.read_parquet("../Data/total data/raw data parquet/reviews_part_02.parquet"),pd.read_parquet("../Data/total data/raw data parquet/reviews_part_03.parquet")])


# EDA

## calendar file

In [ ]:
calendar_total_data.size

In [ ]:
calendar_total_data.dtypes

In [ ]:
calendar_total_data.head()

In [ ]:
print("El fichero tiene "+str(len(calendar_total_data["listing_id"].unique()))+" viviendas")

In [ ]:
calendar_total_data.isna().sum()

In [5]:
from pandas import NA


calendar_total_data_NAs = pd.concat([calendar_total_data[calendar_total_data["price"].isna()],calendar_total_data[calendar_total_data["adjusted_price"].isna()],calendar_total_data[calendar_total_data["minimum_nights"].isna()],calendar_total_data[calendar_total_data["maximum_nights"].isna()]])
calendar_total_data_NAs = calendar_total_data_NAs.drop_duplicates()
calendar_total_data_NAs.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
1421138,5737759,2023-03-16,t,None,None,30.0,180.0
1421139,5737759,2023-03-17,t,None,None,30.0,180.0
1421140,5737759,2023-03-18,t,None,None,30.0,180.0
1421141,5737759,2023-03-19,t,None,None,30.0,180.0
1421142,5737759,2023-03-20,t,None,None,30.0,180.0


In [ ]:
print("Number of rows without data = "+str(len(calendar_total_data_NAs.index.unique())) )

In [ ]:
print("El número de listings que no tienen datos son "+str(len(calendar_total_data_NAs["listing_id"].unique())))
print("Esto representa el "+str(100*len(calendar_total_data_NAs["listing_id"].unique())/len(calendar_total_data["listing_id"].unique()))+"% de los ids")

In [6]:
calendar_total_no_NA = calendar_total_data.dropna()
print("Existen "+str(len(calendar_total_no_NA["listing_id"].unique()))+" viviendas con todos los datos")

Existen 39867 viviendas con todos los datos


In [ ]:
calendar_total_no_NA.head()

In [7]:
calendar_total_no_NA["priceNum"] =calendar_total_no_NA["price"].apply(lambda x: float(x.split()[0].replace('$', '').replace(',','')))
calendar_total_no_NA["adjPriceNum"] = calendar_total_no_NA["adjusted_price"].apply(lambda x: float(x.split()[0].replace('$', '').replace(',','')))

In [8]:
calendar_total_no_NA["date"] = pd.to_datetime(calendar_total_no_NA["date"], errors='coerce')

In [ ]:
calendar_total_no_NA.head()

In [11]:
fig = px.line(data_frame=calendar_total_no_NA[(calendar_total_no_NA["listing_id"]<=40000)], x='date', y='priceNum', color='listing_id')
fig.show()


In [ ]:
calendar_total_no_NA["diff"] = calendar_total_no_NA["priceNum"] - calendar_total_no_NA["adjPriceNum"]

In [ ]:
calendar_total_no_NA.groupby("listing_id",as_index=False).mean("diff")["diff"].describe()